In [ ]:
%load_ext autoreload
%autoreload 2
# %matplotlib widget
%pdb off

from pyCascade.cti import lesCreateImage as lcm
from pyCascade.cti import lesCreateMovie as vcm
import os
from matplotlib import pyplot as plt
import cv2 as cv
import numpy as np

############ Universal ################
scratch_home = os.getenv('SCRATCH') #need to set SCRATCH (even if there is no real SCRATCH) to the location where results are written
scratch_dir = f'{scratch_home}/Cascade/city_block_cfd'
home_dir = !pwd
home_dir = home_dir[0]

display(scratch_dir)
display(home_dir)

In [ ]:
R = 55
config = 2
relRunFolder = f"CHARLES/config{config}/R{R}"

image_path = f"{scratch_dir}/{relRunFolder}/Images/nogeom_u_z58.00240000.png"
output_dir = f"{home_dir}/{relRunFolder}/Images/python_images"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
image_name = os.path.basename(image_path)

varlist=['planar']
cmaplist=['magma']
data_min=[0]
data_max=[3]#[2**2]

# Process the image
image = vcm.process_image(image_path, varlist, cmaplist=cmaplist, data_min=data_min, data_max=data_max)

# Example usage of the reusable function
custom_output = os.path.join(output_dir, image_name)
result = lcm.create_image_with_colorbar(
    image, custom_output, cmaplist, data_min, data_max, 
    cbar_title="Velocity (m/s)",
    cbar_orient="vertical"
)



# Display the result
plt.figure(figsize=(12, 10))
plt.imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))
plt.axis('off')
plt.title(f"{image_name}")
plt.show()

In [ ]:
def collect_image_averages(
    image_name,
    varlist=['planar'], cmaplist=['magma'], data_min=np.array([0]), data_maxs=np.array([[2], [4], [2], [4]]),  data_scale=1,
    crop = None, rotate = 0, steps=[110000, 240000],
    runs = {
        2: {
            "diagonal": [53, 54],# 46, 47],
            "parallel": [55, 56],# 55, 49],
        },
        3: {
            "diagonal": [21, 22],# 16, 18],
            "parallel": [23, 24],# 17, 19],
        }
    },
    ):

    data_maxs = data_maxs / data_scale
    data_min = data_min / data_scale
    im_avgs = {}
    for c, cDict in runs.items():
        for allignment, aList in cDict.items():
            images = []
            for i, R in enumerate(aList):
                relRunFolder = f"CHARLES/config{c}/R{R}"
                output_dir = f"{home_dir}/CHARLES/config{c}/python_images"
                data_max = data_maxs[i]
                for step in steps:
                    image_path = f"{scratch_dir}/{relRunFolder}/Images/{image_name}.00{step}.png"
                    if not os.path.exists(output_dir):
                        os.makedirs(output_dir)

                    # Process the image
                    image = vcm.process_image(image_path, varlist, cmaplist, data_min, data_max)
                    image = np.rot90(image, k=rotate // 90)
                    if crop is not None:
                        shape = image.shape
                        x_start = int(shape[1] * crop[0])
                        x_end = int(shape[1] * crop[1])
                        y_start = int(shape[0] * crop[2])
                        y_end = int(shape[0] * crop[3])

                        image = image[y_start:y_end, x_start:x_end]
                    image = image.astype(np.uint8)
                    images.append(image)

            im_avg = np.mean(images, axis=0)
            im_avgs[f"{c}-{allignment}"] = im_avg
            im_avg = im_avg.astype(np.uint8)
            # Example usage of the reusable function
            custom_output = os.path.join(output_dir, f"{image_name}_{allignment}_avg.png")
            result = lcm.create_image_with_colorbar(
                im_avg, custom_output, cmaplist, [0], [data_scale], 
                cbar_title="Velocity (m/s)",
                cbar_orient="vertical"
            )

            # Display the result
            plt.figure(figsize=(12, 10))
            plt.imshow(cv.cvtColor(result, cv.COLOR_BGR2RGB))
            plt.axis('off')
            # plt.title(f"{image_name}_{allignment}")
            plt.show()

    return im_avgs

data_maxs = np.array([[2], [4], [2], [4]])
# data_maxs = data_maxs**2 / 1.225
data_scale = 2 #4
interiorImages = {}
# name  = "u_avg_h_0--1"
# interiorImages[name] = collect_image_averages(name, cmaplist=['magma'], data_maxs=data_maxs, data_scale=data_scale)
rotations = [0, 90, 180, 270]
for i, block in enumerate(['', '_Bz', '_Bxz', '_Bx']):
    name = f"u_avg_h_0--1{block}"
    interiorImages[name] = collect_image_averages(name, crop = [0, 1, .2, .8], rotate = rotations[i], cmaplist=['magma'], data_maxs=data_maxs, data_scale=data_scale, steps=[110000, 240000],)

In [ ]:
runs = {
        2: {
            "diagonal": [57, 58],
            "parallel": [59, 60],
        },
        3: {
            "diagonal": [25, 26],
            "parallel": [27, 28],
        }
    }

exteriorImages = {}
name  = "p_avg_h_0--1"
exteriorImages[name] = collect_image_averages(name, runs=runs, cmaplist=['cividis'], steps = [119000], data_maxs=data_maxs, data_scale=data_scale)
rotations = [0, 90, 180, 270]
for i, block in enumerate(['', '_Bz', '_Bxz', '_Bx']):
    name = f"p_avg_h_0-0{block}"
    exteriorImages[name] = collect_image_averages(name, crop = [0, 1, .2, .8], rotate = rotations[i], runs=runs, cmaplist=['cividis'], steps = [119000], data_maxs=data_maxs, data_scale=data_scale)

In [ ]:
# Subtract interiorImages from exteriorImages
differenceImages = {}

for key in interiorImages.keys():
    if key in exteriorImages:
        differenceImages[key] = {}
        for subkey in interiorImages[key].keys():
            if subkey in exteriorImages[key]:
                # Perform element-wise subtraction
                differenceImages[key][subkey] = exteriorImages[key][subkey] - interiorImages[key][subkey]

# Display one of the difference images as an example
example_key = list(differenceImages.keys())[0]
example_subkey = list(differenceImages[example_key].keys())[0]

im_diff = differenceImages[example_key][example_subkey]
im_diff = im_diff.astype(np.uint8)
# Example usage of the reusable function
custom_output = None
result = lcm.create_image_with_colorbar(
    im_diff, custom_output, cmaplist, [0], [1], 
    cbar_title="Velocity (m/s)",
    cbar_orient="vertical"
)

# Display the result
plt.figure(figsize=(12, 10))
plt.imshow(cv.cvtColor(result, cv.COLOR_BGR2RGB))
plt.axis('off')
# plt.title(f"{image_name}_{allignment}")
plt.show()

In [ ]:
R = 53
config = 2
relRunFolder = f"CHARLES/config{config}/R{R}"
varlist=['planar']
cmaplist=['seismic']
data_min=[0]
data_max=[4]  # Adjusted for density
data_range = data_max[0] - data_min[0]
normalize = True  # Set to True to normalize by the first image

times = ["110000", "230000"]
images = []
for t in times:
    image_path = f"{scratch_dir}/{relRunFolder}/Images/u_avg_h_0--1_Bz.00{t}.png"
    image = vcm.process_image(image_path, varlist, cmaplist=None, data_min=data_min, data_max=data_max)
    images.append(image)

if normalize:
    # Normalize the second image by the first
    im_mean = images[0]
    im_mean += np.mean(im_mean) / 100  # avoid division by zero
    data_range = 1
else:
    im_mean = 1
im_diff = (images[1] - images[0]) / im_mean # Relative difference
abs_max = np.max(np.abs(im_diff))
# rescale to [0, 1]
im_diff = (im_diff + abs_max) / (2 * abs_max)
data_max = [abs_max * data_range]
data_min = [-abs_max * data_range]

im_diff = plt.get_cmap(cmaplist[0])(im_diff)
im_diff = (im_diff[:,:,:3] * 255).astype(np.uint8)
im_diff = cv.cvtColor(im_diff, cv.COLOR_RGBA2BGR)
# Example usage of the reusable function
custom_output = None
result = lcm.create_image_with_colorbar(
    im_diff, custom_output, cmaplist, data_min, data_max, 
    cbar_title="Velocity (m/s)",
    cbar_orient="vertical"
)

# Display the result
plt.figure(figsize=(12, 10))
plt.imshow(cv.cvtColor(result, cv.COLOR_BGR2RGB))
plt.axis('off')
# plt.title(f"{image_name}_{allignment}")
plt.show()